In [ ]:
import numpy as np
from pyriemann.estimation import XdawnCovariances, Covariances
from pyriemann.tangentspace import TangentSpace
#import mne
#from mne import io
#from mne.datasets import sample
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.base import clone
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import LogisticRegression, SGDRegressor, LinearRegression, ElasticNet, BayesianRidge
from sklearn.svm import LinearSVR, SVR
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.pipeline import make_pipeline
from matplotlib import pyplot as plt
from regression import KNearestNeighborRegression, TSRegressor
import IOHelper
from config import config


: 

In [ ]:
trainX, trainY = IOHelper.get_npz_data(config['data_dir'], verbose=True)


In [ ]:
trainX.shape

(21464, 500, 129)

In [ ]:
trainY = trainY[:,1:]

In [ ]:
sample_indices_channels = np.random.choice(trainX.shape[2], 30, replace=False)
sample_indices_obs = np.random.choice(trainX.shape[0], 2000, replace=False)



In [ ]:
trainX = trainX[:,:,sample_indices_channels]
trainX = trainX[sample_indices_obs]
trainY = trainY[sample_indices_obs]


In [ ]:
estimator = Covariances()
split = train_test_split(trainX, trainY, train_size=0.8, random_state=42)
Xtrain, Xtest, ytrain, ytest = split
Xtrain = np.swapaxes(Xtrain,1,2)
Xtest = np.swapaxes(Xtest,1,2)

Xtrain_covs = estimator.transform(Xtrain)
Xtest_covs = estimator.transform(Xtest)

In [ ]:
knn = KNearestNeighborRegression(n_neighbors=10)
knn.fit(X=Xtrain_covs,y=ytrain)

KNearestNeighborRegression(n_neighbors=10)

In [ ]:
preds = knn.predict(Xtest_covs)
diff = preds - ytest
#error_val_knn = np.sqrt( 1/(preds.shape[0])*np.sum(diff*diff,axis=0))
error_val_knn = [np.sqrt(mean_squared_error(ytest[:,0],preds[:,0])), np.sqrt(mean_squared_error(ytest[:,1],preds[:,1]))]


C:\Users\Guy Berreby\.virtualenvs\Lab_4_Code-yIe1RBd0\lib\site-packages\pyriemann\utils\distance.py:76: RuntimeWarning: invalid value encountered in log
  return np.sqrt((np.log(eigvalsh(A, B))**2).sum())


(200,)

In [ ]:
model_list = [ElasticNet(),BayesianRidge(),SVR(kernel="linear"), SVR(kernel="rbf"), LinearSVR()]
all_model_errors = list()
for model in model_list:

    tanSpace_1 = TSRegressor(clf=clone(model))
    tanSpace_1.fit(Xtrain_covs,ytrain[:,0])
    
    tanSpace_2 = TSRegressor(clf=clone(model))
    tanSpace_2.fit(Xtrain_covs,ytrain[:,1])
    
    
    error_vals_list = list()

    preds = tanSpace_1.predict(Xtest_covs)
    diff = preds - ytest[:,0]
    error_val = np.sqrt(mean_squared_error(ytest[:,0],preds))
    #error_val = np.sqrt( 1/(preds.shape[0])*np.sum(diff*diff,axis=0))
    error_vals_list.append(error_val)
    
    preds = tanSpace_2.predict(Xtest_covs)
    diff = preds - ytest[:,1]
    error_val = np.sqrt(mean_squared_error(ytest[:,1],preds))
    #error_val = np.sqrt( 1/(preds.shape[0])*np.sum(diff*diff,axis=0))
    error_vals_list.append(error_val)

    all_model_errors.append(error_vals_list)

In [ ]:
print("KNN RMS:",error_val_knn)
print("ElasticNet RMS:",all_model_errors[0])
print("Bayseian Ridge RMS:",all_model_errors[1])
print("SVR (Linear) RMS:",all_model_errors[2])
print("SVR (RBF) RMS:",all_model_errors[3])
print("Linear SVR RMS:",all_model_errors[4])


KNN RMS: [248.28592134069947, 151.1955486729024]
ElasticNet RMS: [233.5484143188254, 144.59139310578163]
Bayseian Ridge RMS: [234.11386973894957, 145.16815375115726]
SVR (Linear) RMS: [237.62705174645487, 152.20895293415683]
SVR (RBF) RMS: [234.9647118350431, 145.03204815476394]
Linear SVR RMS: [247.29629126598994, 152.55175833364783]
